In [27]:
import random

import pandas as pd
import numpy as np
import json 
import os
import random
import torch
import tqdm

In [28]:


def concat_txt(dir_path):
    """将原数据集中已经拆分完成的数据还原"""
    rumor_file_suffix = '_rumor.txt'
    nonrumor_file_suffix = '_nonrumor.txt'
    files = os.listdir(dir_path)
    file_prefix = dir_path + '/'
    
    rumor_file = file_prefix + 'rumor.txt'
    nonrumor_file = file_prefix + 'nonrumor.txt'
    for file in files:
        with open(file_prefix + file, 'r') as f:
            if file.endswith(rumor_file_suffix):
                with open(rumor_file, 'a') as f1:
                    f1.write(f.read())
            if file.endswith(nonrumor_file_suffix):
                with open(nonrumor_file, 'a') as f1:
                    f1.write(f.read())

def file2list(file):
    with open(file, 'r') as f:
        return f.readlines()

dir_path = './tweets'
concat_txt(dir_path)
rumor_list = file2list(dir_path + '/rumor.txt')
nonrumor_list = file2list(dir_path + '/nonrumor.txt')

len(rumor_list)//3,len(nonrumor_list)//3

(4748, 4779)

In [29]:
def has_image(imageUrls,image_set):
    """content为三行数据"""
    images = imageUrls.split('|')
    for image in images: # 遍历该新闻相关的图片
        img = image.split('/')[-1]
        if img in image_set:
            return True
    return False



def filter_data(data,image_set):
    result = []
    n_lines = len(data)
    for idx in tqdm.tqdm(range(2, n_lines, 3)):
        text = data[idx].strip()
        if has_image(data[idx-1],image_set) and text:
            result += data[idx-2:idx+1]
    return result

rumor_image_set = os.listdir('./rumor_images/')
nonrumor_image_set = os.listdir('./nonrumor_images/')

rumor_list = filter_data(rumor_list,rumor_image_set)
nonrumor_list = filter_data(nonrumor_list,nonrumor_image_set)
len(rumor_list)//3,len(nonrumor_list)//3

100%|██████████| 4779/4779 [00:00<00:00, 7925.76it/s]


(4211, 3642)

In [30]:
def split_data(data):
    train_lines = 80 * 3
    valid_lines = 80 * 3
    return data[:train_lines] , data[train_lines:train_lines + valid_lines], data[train_lines + valid_lines:]

train_real, valid_real, test_real = split_data(nonrumor_list)
train_fake, valid_fake, test_fake = split_data(rumor_list)
print("train: real {}, fake: {}".format(len(train_real)//3, len(train_fake)//3))
print("valid: real {}, fake: {}".format(len(valid_real)//3, len(valid_fake)//3))
print("test: real {}, fake: {}".format(len(test_real)//3, len(test_fake)//3))


train: real 80, fake: 80
valid: real 80, fake: 80
test: real 3482, fake: 4051


In [31]:

def list2file(file, list):
    with open(file, 'w') as f:
        for item in list:
            f.write(item)


list2file(dir_path + '/train_nonrumor.txt', train_real),list2file(dir_path + '/train_rumor.txt', train_fake)
list2file(dir_path + '/valid_nonrumor.txt', valid_real),list2file(dir_path + '/valid_rumor.txt', valid_fake)
list2file(dir_path + '/test_nonrumor.txt', test_real),list2file(dir_path + '/test_rumor.txt', test_fake)

(None, None)